In [38]:
import nbformat as nbf

In [43]:
nb_id = "toy"
nb = nbf.read(f"{nb_id}.ipynb",as_version=4) #the toy original is the pandas-only control
code = "import lux"
nb["cells"].insert(0,nbf.v4.new_code_cell(code))
nbf.write(nb, f'{nb_id}_baseline.ipynb')

In [44]:
nb = nbf.read(f"{nb_id}.ipynb",as_version=4) 
code = "import lux\nlux.config.lazy_maintain = True"
nb["cells"].insert(0,nbf.v4.new_code_cell(code))
nbf.write(nb, f'{nb_id}_o1.ipynb')

In [45]:
nb = nbf.read(f"{nb_id}.ipynb",as_version=4) 
code = "import lux\nlux.config.lazy_maintain = True\nlux.config.early_pruning = True"
nb["cells"].insert(0,nbf.v4.new_code_cell(code))
nbf.write(nb, f'{nb_id}_o1o2.ipynb')

In [46]:
nb = nbf.read(f"{nb_id}.ipynb",as_version=4) 
code = "import lux\nlux.config.lazy_maintain = True\nlux.config.early_pruning = True\nlux.config.streaming = True"
nb["cells"].insert(0,nbf.v4.new_code_cell(code))
nbf.write(nb, f'{nb_id}_o1o2o3.ipynb')

In [6]:
import glob
import time
import papermill as pm
import pandas as pd
import numpy as np
import json
trial = []
for nb_name in glob.glob("toy*"):
    output_filename = "output.ipynb"
    # papermill basic_scatter.ipynb output.ipynb -p numPoints 1000000 --execute-timeout 1000
    # TODO: High variance in overall time, might have to repeat experiment or measure using papermill nb info
    start = time.perf_counter()
    pm.execute_notebook(
        f"{nb_name}", output_filename, 
    )
    end = time.perf_counter()
    duration = end-start
    print(nb_name, duration)
    trial.append([nb_name, duration])


toy_o1o2.ipynb 27.39405893



toy.ipynb 8.979917790999998



toy_o1.ipynb 36.383189906



toy_o1o2o3.ipynb 12.123425564999991



toy_baseline.ipynb 39.192437846000004


In [9]:
trial = pd.DataFrame(trial,columns=["nb_name","time"])

In [13]:
trial.to_csv("macrobenchmark_1e5.csv",index=None)

In [1]:
import pandas as pd
import altair as alt

In [4]:
trial = pd.read_csv("macrobenchmark_1e6.csv")

In [5]:
alt.Chart(trial).mark_bar().encode(
    x = "nb_name",
    y = "time",
    
)

alt.Chart(...)

In [6]:
df = pd.read_csv("macrobenchmark.csv")

In [7]:
df.nPts = df.nPts.astype(int)

In [8]:
df["condition"] = df.nb_name.replace({"toy_o1.ipynb":"o1",
                    "toy_o1o2o3.ipynb":"o1o2o3",
                    "toy_o1o2.ipynb":"o1o2",
                    "toy.ipynb":"pandas",
                    "toy_baseline.ipynb":"no opt"})

In [9]:
alt.Chart(df).mark_bar().encode(
    x = "condition",
    y = alt.Y("time",scale=alt.Scale(type='log'),title="log(total time)"),
    column="nPts",
    color="condition"
)

alt.Chart(...)

In [10]:
# # VegaLite can not do logarithmic stacked bar since it is misleading
# alt.Chart(df).mark_bar().encode(
#     x = alt.X("nPts",type="ordinal"),
#     y = alt.Y("time"),#,scale=alt.Scale(type='log'),title="log(total time)"),
#     color="condition"
# )

In [26]:
df = df.drop(columns="nb_name")

In [53]:
pdonly = df[df["condition"]=="pandas"]
best = df[df["condition"]=="o1o2o3"]
worst = df[df["condition"]=="no opt"]

In [ ]:
import numpy as np

In [68]:
def pctChange(s1,s2):
    return (np.array(s1["time"])-np.array(s2["time"]))/np.array(s2["time"])

In [69]:
best["time"]

3      33.051949
8      24.736191
13     42.343533
18    197.560367
Name: time, dtype: float64

In [70]:
pctChange(best,pdonly)

array([0.52858007, 0.16597767, 0.74957558, 2.80147911])

In [71]:
pctChange(worst,pdonly)

array([ 3.77364417,  4.78507636, 15.92258916, 53.6757692 ])

In [73]:
pctChange(worst,best)

array([ 2.12292714,  3.96156705,  8.67239677, 13.38276199])

In [19]:

df.groupby(["condition","nPts"])["time"].diff()

,nb_name,time
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN
